In [415]:
%matplotlib
import geopandas as gpd
import glob
from itertools import tee
import networkx as nx
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon, LineString
import matplotlib.pyplot as plt
import matplotlib
import json
import os
from collections import defaultdict
from shapely.geometry import Point, Polygon, LineString

def pairwise(line_coords):
    a,b = tee(line_coords, 2)
    next(b, None)
    return zip(a,b) #return (p1,p2), (p2,p3)...

def create_g(Geo_dataFrame):
    G = nx.DiGraph()
    for line in Geo_dataFrame.geometry:
       seq = pairwise(line.coords)
       for p1, p2 in seq:
           G.add_edge(p1, p2)
    return G

#Euclidean distance
const = np.sqrt(2)
def dist(n0, n1):
    x1, y1 = n0
    x2, y2 = n1
    distance = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

def closest_node(sub_graph, start, end): 
    start_delta = float("inf")
    end_delta = float("inf")
    for n in sub_graph.nodes():
        s_dist = dist(start, n)
        e_dist = dist(end, n) 
        if s_dist < start_delta:
            nn_start = n
            start_delta = s_dist
        if e_dist < end_delta:
            nn_end = n
            end_delta = e_dist
    return (nn_start, nn_end)

def add_attr(sub_graph):
    for n0, n1 in sub_graph.edges_iter():
        distance = dist(n0, n1)
        sub_graph[n0][n1]["dist"] = distance

def shortest_dist(sub_graph, nn_start, nn_end):
    shortest_distance = nx.shortest_path_length(sub_graph, source= nn_start ,target= nn_end, weight= "dist")
    return shortest_distance
def calculate_distance(origin_coords, dest_coords, graph):
    nn_start, nn_end = closest_node(graph, origin_coords, dest_coords)
    shortest_distance = shortest_dist(graph, nn_start, nn_end)
    return shortest_distance

#convert road into graph
Roads = gpd.read_file("/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/AA_City_Version3/N-N_Roads_newline.shp")
G = create_g(Roads)
sg = list(nx.connected_component_subgraphs(G.to_undirected()))[0]
add_attr(sg) #add distance attribute
# Shuttle Route from NCRC to NorthCampus
Route1 = gpd.read_file('/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/AA_city_NAD83/North_South_Route.shp') 
G1 = create_g(Route1)
sg1 = list(nx.connected_component_subgraphs(G1.to_undirected()))[0]
add_attr(sg1)
# Shuttle Route from North Campus to NCRC
Route2 = gpd.read_file('/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/AA_city_NAD83/South_North_Route.shp')
G2 = create_g(Route2)
sg2 = list(nx.connected_component_subgraphs(G2.to_undirected()))[0]
add_attr(sg2)

# bus_stops, from NCRC to NorthCampus
stop1 = (276980.0661116603, 4686804.277913463)
stop2 = (276660.2629301893, 4685910.528365758)
stop3 = (276278.1061585448, 4685419.189922427)       # Lurie
stop4 = (276114.8522941918, 4685676.255754295)
stop_dict = {'stop1':stop1,'stop2':stop2,'stop3':stop3,'stop4':stop4}
# find nearest bus-stop among four stops
def nearest(n0):     # find nearest bus-stop of 4stops
    x0, y0 = n0
    d1 = dist(n0, stop1)
    d2 = dist(n0, stop2)
    d3 = dist(n0, stop3)
    d4 = dist(n0, stop4)
    if d1 == min(d1, d2, d3, d4):return str('stop1')
    elif d2 == min(d1, d2, d3, d4):return str('stop2')
    elif d3 == min(d1, d2, d3, d4):return str('stop3')
    elif d4 == min(d1, d2, d3, d4):return str('stop4')

Using matplotlib backend: MacOSX


In [416]:
import geopandas as gpd
import pandas as pd
import numpy as np

Building = pd.DataFrame.from_csv('/Users/mac/Desktop/utm_test/Buildings.csv') # building frequency according to wifi-data      
Buildings = gpd.read_file("/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/AA_city_NAD83/N-N_Buildings.shp")
eudist = pd.DataFrame.from_csv('/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/network_csv/eudist.csv') 
lat_line = 290560            # lat_line between NCRC & North Campus
const = np.sqrt(2)

In [417]:
# create frequency dataframe of all buildings,including NCRC
NCRC_frequency = list(map(int, np.random.normal(loc=291.0, scale=20, size=21)))    # mean:291(UMTRI)
Building.loc[(Building['Latitude']>lat_line)&(Building.OBJECTID_1!=91)&(Building.OBJECTID_1!=61)&(Building.OBJECTID_1!=79),'frequency'] = NCRC_frequency
Building.loc[48,'frequency'] = 291
# find nearest stop of each building
Building['nearest_stop'] = pd.Series(0)
for bld in Building.index:
    Building.loc[bld,'nearest_stop'] = nearest(Buildings.geometry[bld].coords[0])
frequency = Building.frequency.tolist()  # list of frequency of all buildings
ID = Building.OBJECTID_1.tolist()          # list of index of all buildings
# create dataframe of probability from given Oringin to Destination
P = pd.DataFrame(0,index = ID, columns = eudist.columns)
i = 0
while i<len(ID):
    tmp = str(ID[i])
    P[tmp] = P[tmp] + eudist[tmp]*frequency[i]
    i = i+1
for Oid in P.index:
    stop = Building[Building.OBJECTID_1==Oid]['nearest_stop'].tolist()[0]
    for id_tmp in Building[Building['nearest_stop']==stop].OBJECTID_1.tolist():
        P.loc[Oid,str(id_tmp)]=0
for j in P.index:
    P.loc[j,:] = P.loc[j,:]/sum(P.loc[j,:])
# probability of origins
P_Origin = list(map(lambda x:x/sum(frequency), frequency))

In [424]:
# random generating some trips
N = 100   # number of trips
trip = pd.read_csv('/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/Trip/allTRIP/random trip/random trip.csv')
trip['trip_id'] = range(N)
# random generating origins
import random
from numpy.random import choice
O_list = []
i = 0
while i < N:
    O_list.append(choice(ID,1,False, P_Origin)[0])
    i = i + 1
# random generating destinations of given origin
D_list = []
j = 0
while j < N:
    D_list.append(choice(ID,1,False,P.loc[O_list[j],:])[0])
    j = j + 1
# random start time of trip
import time
import datetime
starttime_list = []
t_upper = datetime.datetime(2017,7,14,15,45,0)
t_lower = datetime.datetime(2017,7,14,16,45,0)
unixtime_upper = time.mktime(t_upper.timetuple())
unixtime_lower = time.mktime(t_lower.timetuple())
k = 0
while k < N:
    tmp = random.randrange(unixtime_upper, unixtime_lower)
    starttime_list.append(datetime.datetime.fromtimestamp(tmp))
    k = k + 1
# random parking time of each OD_pairs
import scipy.stats
lower = 0
upper = 300
mu = 90         # average parking time, unit of sec
sigma = 70      # variance parking time          
parkingtime_list = scipy.stats.truncnorm.rvs(
          (lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=N).tolist()

In [425]:
# filling the dataframe
trip['O_id'] = O_list; trip['D_id'] = D_list; trip['start_t'] = starttime_list
for i in trip.index:
    O_index = ID.index(trip.loc[i,'O_id']);D_index = ID.index(trip.loc[i,'D_id'])
    trip.loc[i,'O_lon'] = Buildings.geometry[O_index].coords[0][0]
    trip.loc[i,'O_lat'] = Buildings.geometry[O_index].coords[0][1]
    trip.loc[i,'D_lon'] = Buildings.geometry[D_index].coords[0][0]
    trip.loc[i,'D_lat'] = Buildings.geometry[D_index].coords[0][1]
    O = trip.loc[i,'O_lon'],trip.loc[i,'O_lat'];D = trip.loc[i,'D_lon'],trip.loc[i,'D_lat']
    trip.loc[i,'O_stop'] = nearest(O); trip.loc[i,'D_stop'] = nearest(D)
    O_stop = stop_dict[trip.loc[i,'O_stop']];D_stop = stop_dict[trip.loc[i,'D_stop']]
    trip.loc[i,'bus_t'] = trip.loc[i,'start_t']+datetime.timedelta(0,int(const*dist(O,O_stop)/1.5))
    if int(trip.loc[i,'O_stop'][4])>int(trip.loc[i,'D_stop'][4]):
        trip.loc[i,'shuttletrip_t'] = int(const*(dist(O,O_stop)+dist(D,D_stop))/1.5+calculate_distance(O_stop,D_stop,sg1)/11)
    else:trip.loc[i,'shuttletrip_t'] = int(const*(dist(O,O_stop)+dist(D,D_stop))/1.5+calculate_distance(O_stop,D_stop,sg2)/11)
    trip.loc[i,'drivetrip_sec'] = int(calculate_distance(O, D, sg)/11 + parkingtime_list[i])

In [426]:
trip

,trip_id,O_id,O_lon,O_lat,D_id,D_lon,D_lat,O_stop,D_stop,start_t,bus_t,shuttletrip_t,drivetrip_sec
0,0,103,276810,4.68672e+06,40,276268,4.68571e+06,stop1,stop4,2017-07-14 16:30:04,2017-07-14 16:33:03,378,179
1,1,108,276885,4.68665e+06,9,276430,4.68593e+06,stop1,stop2,2017-07-14 16:35:23,2017-07-14 16:38:10,438,191
2,2,41,276274,4.68566e+06,77,276751,4.68663e+06,stop4,stop1,2017-07-14 16:30:01,2017-07-14 16:32:31,440,181
3,3,51,276229,4.68582e+06,29,276278,4.68542e+06,stop4,stop3,2017-07-14 16:18:29,2017-07-14 16:21:18,170,177
4,4,47,276534,4.6859e+06,99,276923,4.68655e+06,stop2,stop1,2017-07-14 16:44:21,2017-07-14 16:46:20,381,105
5,5,37,276073,4.68588e+06,3,276662,4.68587e+06,stop4,stop2,2017-07-14 16:41:30,2017-07-14 16:44:42,232,294
6,6,53,276269,4.6859e+06,97,276814,4.6864e+06,stop4,stop1,2017-07-14 16:30:56,2017-07-14 16:35:13,684,302
7,7,74,276995,4.68654e+06,51,276229,4.68582e+06,stop1,stop4,2017-07-14 16:31:20,2017-07-14 16:35:33,474,208
8,8,67,276733,4.68678e+06,43,276279,4.68561e+06,stop1,stop4,2017-07-14 16:43:07,2017-07-14 16:47:01,451,290
9,9,67,276733,4.68678e+06,49,275966,4.68568e+06,stop1,stop4,2017-07-14 15:53:13,2017-07-14 15:57:07,425,430
